# Paper Data Processing

## Mesh to SDF

In [43]:
# The path to the mesh file
meshPath = r"data\GroteSilo_Downsample.obj"
saveFolder = r"data"

voxelGridSize = 64

In [44]:
import trimesh
import os
import comptools
from alphashape import alphashape
mesh = trimesh.load(meshPath)
meshName = meshPath.split('\\')[-1][:-4]
savePath = os.path.join(saveFolder, meshName)
comptools.make_dir_if_not_exist(savePath)

### Display the mesh (Optional)

In [ ]:
import math
import trimesh.transformations as transformations

# Recenter the mesh to the origin
mesh.apply_translation(-mesh.bounding_box.centroid)

# Normalize the scale of the mesh
# Scale so the bounding box fits within a unit cube
bounding_box_extents = mesh.bounding_box.extents
scale_factor = 1.0 / max(bounding_box_extents)
mesh.apply_scale(scale_factor)

angle = -math.pi/2
#direction = [1, 0, 0]
direction = [0, 1, 0]
center = [0, 0, 0]

rot_matrix = transformations.rotation_matrix(angle, direction, center)

mesh.apply_transform(rot_matrix)
mesh.show()

In [ ]:
#Creating a alpha shape to create a more complex hull
alphaValue = 2
alphaMesh = alphashape(mesh.vertices, alphaValue)
#alphaMesh.invert()
alphaMesh.show()

In [57]:
mesh = alphaMesh

### SDF Creation

In [ ]:
import torch
import numpy as np
import time

t0 = time.time() # keep track of how long the process takes
sdf,sdfMesh = comptools.mesh_to_sdf_tensor(comptools.as_mesh(mesh), voxelGridSize, recenter=True, scaledownFactor=8)

fixedMeshPath = os.path.join(savePath,"sdf.obj")
sdfPath = fixedMeshPath[:-4] + ".npy"
tfPath = fixedMeshPath[:-4] + ".pt"

# output
mesh.export(os.path.join(savePath, "gt.obj"))
sdfMesh.export(fixedMeshPath)
np.save(sdfPath, sdf)
# Tensor
shapedData = np.reshape(sdf, [1, 1, 64, 64, 64])
torch.save(torch.from_numpy(shapedData[:,:,:64,:64,:64]), tfPath)

t1 = time.time()
print('It takes %.4f seconds to process' % (t1-t0))
sdfMesh.show()